## Import All Required Libraries

In [1]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql
import psycopg2
import requests

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [2]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Establish External Connections

In [3]:
# establish connection to Spotify API

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = 'aaaa11ed71604ff19d592070be0f6bbd'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

## Load Initial Training Data

In [4]:

# load training data
# new_batch = pd.read_csv('distinct_playlists.csv')[['playlist_uri', 'playlist_name']]
# new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

# db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

# outer = new_batch.merge(db_query, how='outer', indicator=True)
# anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

# new_batch = pd.DataFrame(anti_join)
# new_batch = new_batch[9:29]
# new_batch.head()

## Iterate Through Training Data to Change Grain of Data

In [5]:
# load_batch = []
# series = new_batch['playlist_uri'].to_dict()
# for playlist in tqdm(series.values()):
#     try:
#         tracks = get_playlist_tracks(playlist)
#         if len(tracks) >= 100:
#                 tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
#                 tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
#                 tracks = pd.concat([tracks, tracks2, tracks3])
#                 secondary_data = pd.json_normalize(sp.playlist(playlist))
#                 tracks['playlist_uri'] = playlist
#                 tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
#                 tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
#                 tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

#                 load_batch.append(tracks)
#         else:
#                 secondary_data = pd.json_normalize(sp.playlist(playlist))
#                 tracks['playlist_uri'] = playlist
#                 tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
#                 tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
#                 tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
#                 load_batch.append(tracks)
#     except:
#             pass
# load_batch = pd.concat(load_batch)
# load_batch.head()

## Upload Training Data to playlist_tracks Table

In [6]:
#load_batch.to_sql('playlist_tracks', engine, if_exists='append')

## Identify All Artists in Every Song

In [7]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM artists''', engine)
df2 = pd.read_sql('''SELECT DISTINCT track_uri FROM audio_features''', engine)
outer = df2.merge(df, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

df = pd.Series(anti_join['track_uri'].head(2000))
base_list = []
for track in tqdm(df):
    try:
        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
        df2['track_uri'] = track
        df2 = df2[['name', 'track_uri']]
        base_list.append(df2)
    except:
        pass
df2 = pd.concat(base_list)
df2

  0%|          | 0/2000 [00:00<?, ?it/s]

## Get Row Numbers for Pivoting

In [ ]:
df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
df2

,name,track_uri,RN
0,Laura Hessler,spotify:track:3MIpdWQxTgqN5JrDuZ9qdz,1.0
0,Charley Pride,spotify:track:68oAFaDNBlWVrvilWRjYUC,1.0
1,The Nashville Edition,spotify:track:68oAFaDNBlWVrvilWRjYUC,2.0
0,Ancira,spotify:track:4n1x81F6TqZshFhrZiDHwa,1.0
0,RAYE,spotify:track:2MdoVJfbVNOsZx9AQ7Eqjh,1.0
...,...,...,...
1,zakè,spotify:track:65v54Xx1QYM4FqOUxbIguB,2.0
0,George Strait,spotify:track:47HqQyf0n30Gv2XLcYPutT,1.0
0,Black Eyed Peas,spotify:track:26shgpvmOpoAiXjcuKCyJw,1.0
0,Radio And Weasel,spotify:track:1ck6aiQHeUxzCn3oNtOgSN,1.0


## Pivot DataFrame to Fit all Artists in DataFrame

In [ ]:
df2 = df2.pivot(index='track_uri', columns=['RN'], values='name').reset_index()
df2 = df2.rename(columns={1.0:'artist1', 2.0:'artist2', 3.0:'artist3', 4.0:'artist4', 5.0:'artist5'})
df2 = df2[['track_uri', 'artist1', 'artist2', 'artist3', 'artist4', 'artist5']]
df2

RN,track_uri,artist1,artist2,artist3,artist4,artist5
0,spotify:track:00pg6ROBxCsQ8lL9fpmosR,Justin Bieber,Omah Lay,NaN,NaN,NaN
1,spotify:track:00t7QTffOR3SA3L1BvSQVq,The Notorious B.I.G.,NaN,NaN,NaN,NaN
2,spotify:track:016blcFaFbzXopGoqVOKZU,Eden Hason,NaN,NaN,NaN,NaN
3,spotify:track:01GORqErvC1GnNH0QiH0F1,Empas,Sebastian Wojkowski,NaN,NaN,NaN
4,spotify:track:01TCjbRSrpKwH1pAMA3wzK,CAIN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1995,spotify:track:7yN9Qx1HtE4c1fhakBz8Ay,Damso,NaN,NaN,NaN,NaN
1996,spotify:track:7zBq7YCjuIxQRxqhtKJNbI,Aly & AJ,NaN,NaN,NaN,NaN
1997,spotify:track:7zRoARD3qujVfr811sW2u1,Armin van Buuren,Garibay,NaN,NaN,NaN
1998,spotify:track:7zfDs9VH4Ib790gpeZrohY,Six Market Blvd.,NaN,NaN,NaN,NaN


## Upload All Artist Data to Database

In [ ]:
df2.to_sql('artists', con=engine, if_exists='append')

1000

## Get All Audio Features

In [ ]:
# df = pd.read_sql('''SELECT DISTINCT track_uri FROM audio_features''', engine)
# outer = load_batch.merge(df, how='outer', indicator=True)
# anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
# df = pd.Series(anti_join['track_uri'].head(5000))

# base_list = []
# for track in tqdm(df):
#     try:
#         df3 = pd.json_normalize(sp.audio_features(track))
#         df3['track_uri'] = track
#         df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
#         base_list.append(df3)
#     except:
#         pass
# df3 = pd.concat(base_list)

## Upload Audio Features to Database

In [ ]:
#df3.to_sql('audio_features', con=engine, if_exists='append')

In [ ]:
all_data = pd.read_sql('''SELECT DISTINCT playlist_uri, playlist_name, owner_uri, owner_display_name, collaborative, description, followers_total,
af.track_uri, track_name, artist1, artist2, artist3, artist4, artist5, album_uri, album_name, release_date, 
added_by_uri, added_at, track_album_total_tracks, af.duration_ms, "isExplicit", track_popularity, 
track_preview_url, video_thumbnail_url, danceability, energy, loudness, key, mode, acousticness, speechiness, 
instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN artists a ON pt.track_uri = a.track_uri
JOIN audio_features af ON pt.track_uri = af.track_uri;''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

all_data.to_csv('s3.csv', mode="w+")